## What is PyMongo?

PyMongo is the official Python driver for MongoDB.

- It allows Python applications to talk directly to MongoDB
- No ORM, no abstraction
- You work with MongoDB exactly as it is

Think of PyMongo as:
> "SQLAlchemy Core for MongoDB"


In [1]:
# Run this in terminal, not inside notebook
# pip install pymongo


### Make sure MongoDB is running

Start MongoDB:
Open Mongo shell:
```mongosh```

In [1]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["school"]
students_col = db["students"]

print("Connected to MongoDB")


Connected to MongoDB


### Insert ONE document

In [3]:
student = {
    "name": "Rahul",
    "age": 22,
    "course": "Data Engineering"
}

result = students_col.insert_one(student)
print("Inserted ID:", result.inserted_id)


Inserted ID: 69817bc37ee9800157c6dc61


### Insert MANY documents

In [2]:
students = [
    {"name": "Anita", "age": 21, "course": "AI"},
    {"name": "Kiran", "age": 23, "course": "Big Data"},
    {"name": "Meena", "age": 22, "course": "AI"}
]

students_col.insert_many(students)
print("Multiple students inserted")


Multiple students inserted


### Find ALL documents

In [5]:
for doc in students_col.find():
    print(doc)


{'_id': ObjectId('69817bc37ee9800157c6dc61'), 'name': 'Rahul', 'age': 22, 'course': 'Data Engineering'}
{'_id': ObjectId('69817bce7ee9800157c6dc62'), 'name': 'Anita', 'age': 21, 'course': 'AI'}
{'_id': ObjectId('69817bce7ee9800157c6dc63'), 'name': 'Kiran', 'age': 23, 'course': 'Big Data'}
{'_id': ObjectId('69817bce7ee9800157c6dc64'), 'name': 'Meena', 'age': 22, 'course': 'AI'}


### Find with FILTER

In [6]:
for doc in students_col.find({"course": "AI"}):
    print(doc)


{'_id': ObjectId('69817bce7ee9800157c6dc62'), 'name': 'Anita', 'age': 21, 'course': 'AI'}
{'_id': ObjectId('69817bce7ee9800157c6dc64'), 'name': 'Meena', 'age': 22, 'course': 'AI'}


### Find ONE document

In [7]:
student = students_col.find_one({"name": "Rahul"})
print(student)


{'_id': ObjectId('69817bc37ee9800157c6dc61'), 'name': 'Rahul', 'age': 22, 'course': 'Data Engineering'}


### Projection (select fields)

In [11]:
for doc in students_col.find(
    {"course": "AI"},
    {"_id": 0, "name": 1, "age": 1}
):
    print(doc)


{'name': 'Anita', 'age': 21}
{'name': 'Meena', 'age': 22}


### Update ONE document ($set)

In [12]:
students_col.update_one(
    {"name": "Rahul"},
    {"$set": {"age": 23}}
)

print("Age updated")


Age updated


### Increment a field ($inc)

In [13]:
students_col.update_one(
    {"name": "Rahul"},
    {"$inc": {"age": 1}}
)

print("Age incremented")


Age incremented


### Delete ONE document

In [14]:
students_col.delete_one({"name": "Rahul"})
print("Rahul deleted")


Rahul deleted


### Delete MANY documents

In [15]:
students_col.delete_many({"course": "AI"})
print("All AI students deleted")


All AI students deleted


### Count documents

In [16]:
print("Total students:", students_col.count_documents({}))
print("Big Data students:", students_col.count_documents({"course": "Big Data"}))


Total students: 1
Big Data students: 1


### Sorting & Limiting

In [17]:
for doc in students_col.find().sort("age", -1).limit(2):
    print(doc)


{'_id': ObjectId('69817bce7ee9800157c6dc63'), 'name': 'Kiran', 'age': 23, 'course': 'Big Data'}


### ObjectId (VERY IMPORTANT)

In [18]:
from bson import ObjectId

doc = students_col.find_one()
print("Original ID:", doc["_id"])

# Find using ObjectId
students_col.find_one({"_id": ObjectId(str(doc["_id"]))})


Original ID: 69817bce7ee9800157c6dc63


{'_id': ObjectId('69817bce7ee9800157c6dc63'),
 'name': 'Kiran',
 'age': 23,
 'course': 'Big Data'}

### Index (Performance basics)

In [19]:
students_col.create_index("course")
print("Index created on course")


Index created on course


### Real-world example: Inventory

In [20]:
inventory_col = db["inventory"]

inventory_col.insert_many([
    {"name": "Laptop", "price": 50000, "quantity": 5},
    {"name": "Mouse", "price": 500, "quantity": 20}
])

for item in inventory_col.find():
    print(item)


{'_id': ObjectId('69817d647ee9800157c6dc65'), 'name': 'Laptop', 'price': 50000, 'quantity': 5}
{'_id': ObjectId('69817d647ee9800157c6dc66'), 'name': 'Mouse', 'price': 500, 'quantity': 20}


### Purchase product (Atomic update)

In [21]:
result = inventory_col.find_one_and_update(
    {
        "name": "Laptop",
        "quantity": {"$gte": 1}
    },
    {"$inc": {"quantity": -1}},
    return_document=True
)

if result:
    print("Purchase successful, remaining:", result["quantity"])
else:
    print("Out of stock")


Purchase successful, remaining: 4


### Cleanup (optional)

In [22]:
# WARNING: deletes all data
students_col.delete_many({})
inventory_col.delete_many({})

print("Collections cleaned")


Collections cleaned
